In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from ipywidgets import interact, Dropdown, IntSlider, DatePicker

def reorder_csv(csv_file):
    df = pd.read_csv(csv_file)
    df['predict_time'] = pd.to_datetime(df['predict_time'], utc=True).dt.tz_convert('US/Pacific')
    df['forecast_time'] = pd.to_datetime(df['forecast_time'], utc=True).dt.tz_convert('US/Pacific')
    forecast_lookup = {}
    for f_time, group in df.groupby('forecast_time'):
        forecast_lookup[f_time] = group.sort_values('predict_time')
    return df, forecast_lookup


def get_latest_forecast_value(forecast_lookup, variable, current_time, target_time):
    if target_time not in forecast_lookup:
        return np.nan
    group = forecast_lookup[target_time]
    candidates = group[group['predict_time'] <= current_time]
    if candidates.empty:
        return np.nan
    return candidates.iloc[-1][variable]

def generate_forecast_vector(forecast_lookup, variable, current_time, horizon):
    current_time = pd.to_datetime(current_time)
    forecast_vector = []
    for h in range(1, horizon + 1):
        target_time = current_time + pd.Timedelta(hours=h)
        value = get_latest_forecast_value(forecast_lookup, variable, current_time, target_time)
        forecast_vector.append(value)
    return np.array(forecast_vector)

def generate_forecasts_over_time(forecast_lookup, variable, horizon):
    assert horizon <= 119, "Horizon must be <= 119 hours"
    start_time = pd.Timestamp('2018-01-01 00:00:00-08:00', tz='US/Pacific')
    end_time = pd.Timestamp('2022-12-31 23:00:00-08:00', tz='US/Pacific')
    tz = start_time.tz
    hourly_times = pd.date_range(start=start_time, end=end_time, freq='h', tz=tz)
    forecast_dict = {}
    for current_time in hourly_times:
        vec = generate_forecast_vector(forecast_lookup, variable, current_time, horizon)
        forecast_dict[current_time] = vec
    return forecast_dict

def forecast_dict_to_dataframe(forecast_dict):
    df = pd.DataFrame.from_dict(forecast_dict, orient='index')
    df.index.name = 'prediction_time'
    df.columns = [f'h+{i}' for i in range(1, df.shape[1] + 1)]
    return df

def launch_day_hour_forecast_picker(forecast_df):
    """
    Interactive line plot using a calendar (for day) and a slider (for hour).
    """
    available_times = forecast_df.index.tz_convert('US/Pacific')
    available_dates = sorted(set(ts.date() for ts in available_times))

    # Global y-axis scale
    global_min = forecast_df.min().min()
    global_max = forecast_df.max().max()

    @interact(
        date=DatePicker(
            value=available_dates[len(available_dates)//2],
            description='Date:'
        ),
        hour=IntSlider(min=0, max=23, step=1, value=12, description='Hour:')
    )
    def plot_forecast(date, hour):
        # Combine selected date and hour into a timezone-aware datetime
        try:
            selected_dt = pd.Timestamp(f'{date} {hour:02d}:00:00', tz='US/Pacific')
        except Exception:
            print("Invalid date/hour selection.")
            return

        if selected_dt not in forecast_df.index:
            print("No forecast available at selected date and hour.")
            return

        y = forecast_df.loc[selected_dt]
        x = range(1, len(y) + 1)

        plt.figure(figsize=(8, 4))
        plt.plot(x, y, color='black', linewidth=2)
        plt.title(f'Forecast issued at {selected_dt.tz_convert("US/Pacific").strftime("%Y-%m-%d %H:%M %Z")}')
        plt.xlabel('Forecast Horizon (hours ahead)')
        plt.ylabel('Forecasted Value')
        plt.ylim(global_min, global_max)
        plt.grid(True)
        plt.tight_layout()
        plt.show()

In [3]:
df_reordered, forecast_lookup = reorder_csv('forecasts.csv')
df_hat_temp_72 = forecast_dict_to_dataframe(generate_forecasts_over_time(forecast_lookup, 'temperature', 72))

In [4]:
launch_day_hour_forecast_picker(df_hat_temp_72)

interactive(children=(DatePicker(value=datetime.date(2020, 7, 2), description='Date:', step=1), IntSlider(valu…